In [1]:
%reset -f
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import sys
sys.path.insert(0,'..')
from wavhandler import *
import numpy as np
import sys
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from utils_train import train_test_val_split, TrainConfiguration, train_generator, valid_generator,mosquito_data_split, train_model_dl
from utils_train import n_cpus
import matplotlib.pyplot as plt
import seaborn as sns
import os
seed = 42
np.random.seed(seed=seed)

In [2]:
splitting = 'randomcv'
data_setting = 'raw'
model_setting = 'conv1d'

assert splitting in ['random','randomcv','custom'], "Wrong splitting method given."
assert data_setting in ['raw','stft','psd_dB'], "Wrong data settting given."
assert model_setting in ['wavenet','lstm','gru','conv1d','conv1d_psd',
                        'DenseNet121','DenseNet169','DenseNet201',
                        'InceptionResNetV2','VGG16','VGG19'], "Wrong model setting given"

data = Dataset('Wingbeats')
print(data.target_classes)

print(f'SPLITTING DATA {splitting}')
X_train, X_val, X_test, y_train, y_val, y_test = mosquito_data_split(splitting=splitting, dataset=data)

['Ae. aegypti', 'Ae. albopictus', 'An. arabiensis', 'An. gambiae', 'C. pipiens', 'C. quinquefasciatus']
SPLITTING DATA randomcv
Species: Ae. aegypti.
Read 85553 filenames in 2.38 seconds.
['20170209', '20170208', '20170210', '20170206', '20170213', '20170211', '20170207', '20161213', '20170212', '20161212']
82989 train filenames, 2564 test filenames
Species: Ae. albopictus.
Read 20231 filenames in 2.37 seconds.
['20170301', '20170228', '20170227', '20170103', '20170102']
17440 train filenames, 2791 test filenames
Species: An. arabiensis.
Read 19297 filenames in 2.36 seconds.
['20170313', '20170202', '20170315', '20170314', '20170318', '20170204', '20170201', '20170131', '20170203', '20170317', '20170320', '20170319', '20170316', '20170205', '20170206', '20170130']
12897 train filenames, 2831 test filenames
Species: An. gambiae.
Read 49471 filenames in 2.27 seconds.
['20170110', '20170109', '20170116', '20170119', '20170120', '20170117', '20170121', '20170118', '20170122', '20170123']
4

In [3]:
from utils_train import ModelConfiguration, TrainConfiguration

traincf = TrainConfiguration(dataset=data, 
                             setting=data_setting,
                             batch_size=64,
                             model_name=f'{splitting}_{data_setting}_{model_setting}_ss')

In [4]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [ ]:
encoding_dim = 2
input_img = Input(batch_shape=(None,5000,1))
# encoder = Conv1D(256,3, activation='relu', padding='same')(input_img)
# encoder = Conv1D(256,3, activation='relu', padding='same')(encoder)
# encoder = MaxPooling1D(2)(encoder)
# encoder = Conv1D(128,3, activation='relu', padding='same')(input_img)
# encoder = Conv1D(128,3, activation='relu', padding='same')(encoder)
# encoder = MaxPooling1D(2)(encoder)
encoder = Conv1D(64,3, activation='relu', padding='same')(input_img)
encoder = Conv1D(64,3, activation='relu', padding='same')(encoder)
encoder = MaxPooling1D(2)(encoder)
encoder = Conv1D(32,3, activation='relu', padding='same')(encoder)
encoder = Conv1D(32,3, activation='relu', padding='same')(encoder)
# encoder = BatchNormalization()(encoder)
encoder = MaxPooling1D(2)(encoder)
encoder = Conv1D(16,3, activation='relu', padding='same')(encoder)
encoder = Conv1D(16,3, activation='relu', padding='same')(encoder)
encoder = MaxPooling1D(2)(encoder)

encoder = Dense(encoding_dim, activation='relu')(encoder)

decoder = Conv1D(16, 3, activation='relu', padding='same')(encoder)
decoder = Conv1D(16, 3, activation='relu', padding='same')(decoder)
decoder =  UpSampling1D(2)(decoder)
decoder = Conv1D(32, 3, activation='relu', padding='same')(decoder)
decoder = Conv1D(32, 3, activation='relu', padding='same')(decoder)
decoder =  UpSampling1D(2)(decoder)
# decoder = BatchNormalization()(decoder)
decoder = Conv1D(64, 3, activation='relu', padding='same')(decoder)
decoder = Conv1D(64, 3, activation='relu', padding='same')(decoder)
decoder =  UpSampling1D(2)(decoder)
# decoder = Conv1D(128, 3, activation='relu', padding='same')(decoder)
# decoder = Conv1D(128, 3, activation='relu', padding='same')(decoder)
# decoder =  UpSampling1D(2)(decoder)
# decoder = Conv1D(256, 3, activation='relu', padding='same')(decoder)
# decoder = Conv1D(256, 3, activation='relu', padding='same')(decoder)
# decoder =  UpSampling1D(2)(decoder)
decoder = Conv1D(1, 3, activation='sigmoid', padding='same')(decoder)
autoencoder = Model(input_img, decoder)
autoencoder.compile(optimizer='adam', loss='mse')
print(autoencoder.summary())

In [ ]:
x_train = make_df_parallel(names=X_train, setting='raw')

In [ ]:
x_train = np.expand_dims(x_train.values, axis=-1)

In [ ]:
x_test = make_df_parallel(names=X_test, setting='raw')
x_test = np.expand_dims(x_test.values, axis=-1)

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=1,
                batch_size=16,
                shuffle=True,
                validation_data=(x_test, x_test))
#                 callbacks=[TensorBoard(log_dir='../temp_data/logs/fit/', histogram_freq=0, write_graph=False)])